In [2]:
from dataclasses import dataclass
from pathlib import Path
import kagglehub
import sys
import os

/Users/lakitoshsingh/Personal/E2E-ML-Project/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    kaggle_dataset: str
    local_data_file: Path
    unzip_path: Path
    

In [3]:
%pwd

'/Users/lakitoshsingh/Personal/E2E-ML-Project/research'

In [4]:
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
sys.path.append(os.path.abspath(os.path.join('..', 'config')))

In [9]:
%pwd

'/Users/lakitoshsingh/Personal/E2E-ML-Project/research'

In [8]:
from mlproject.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from mlproject.utils.common import read_yaml, create_directories
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        print(f"Using config file path: {config_filepath}")
        print(f"Using params file path: {params_filepath}")
        print(f"Using schema file path: {schema_filepath}")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
       
    
    
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            kaggle_dataset=config.kaggle_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
        

ModuleNotFoundError: No module named 'mlproject'

In [27]:
import os
from urllib import request
import zipfile
from mlproject import logger
from mlproject.utils.common import get_size

In [28]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            url = kagglehub.dataset_download(self.config.kaggle_dataset)
            filename, headers = request.urlretrieve(
                url=url, 
                filename=self.config.local_data_file
            )
            
            logger.info(f"{filename} has been downloaded with following info {headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_path
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        
            

In [1]:
# pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

NameError: name 'ConfigurationManager' is not defined

In [10]:
print(os.getcwd())

/Users/lakitoshsingh/Personal/E2E-ML-Project/research
